# Сбор новостей с сайтов Yandex.ru, Mail.ru, Lenta.ru

In [ ]:
from pprint import pprint
from lxml import html
import requests
import re
import datetime
import json

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

In [ ]:
def request_to_yandex():
    response = requests.get('https://yandex.ru/news/', headers=header)
    dom = html.fromstring(response.text)

    news = []
    items = dom.xpath("//div[@class='stories-set stories-set_main_no stories-set_pos_1']//td")
    for item in items:
        new = {}
        source1 = item.xpath(".//div[@class='story__info']/div/text()")
        source_news = re.findall(r'([^\d+:\d+]+|^\w+\.+\w+)', source1[0])
        name = item.xpath(".//h2/a/text()")
        link = item.xpath(".//h2/a/@href")
        time_news = re.findall(r'\d+:\d+', source1[0])
        date_news = datetime.datetime.today().strftime('%d/%m/%Y')

        if ' вчера\xa0в\xa0' in source_news: #Если новость вчерашняя вычетаем один день из даты
            a = date_news.split('/')
            d = int(a[0]) - 1
            m = int(a[1])
            y = int(a[2])
            date_news = datetime.date(y, m, d).strftime('%d/%m/%Y')
            date_time_news = date_news + ' ' + time_news[0] +' (Вчера)'#В выводе добавляем пометку (Вчера)
            source_news.pop(1)
        else:
            date_time_news = date_news + ' ' + time_news[0]

        new['link'] = link[0]
        new['date'] = date_time_news
        new['name'] = name[0]
        new['source'] = source_news[0]
        news.append(new)
        with open('news_yandex.json', 'w') as f:
            json.dump(news, f)

    return news

In [ ]:
def request_to_mailru():
    response = requests.get('https://sportmail.ru/',
                            headers=header)
    dom = html.fromstring(response.text)
    news = []
    items = dom.xpath("//div[@class='block']//li")
    for item in items:
        new = {}
        name = item.xpath("./a/text()")
        link = item.xpath("./a/@href")
        new['name'] = name[0].replace('\xa0', ' ')
        new['link'] = link[0]
        response1 = requests.get(link[0], headers=header) #Делаем Get-запрос по ссылке новости,                                      
        dom1 = html.fromstring(response1.text)            #чтобы "вытянуть" дату и источник новости.
        items1 = dom1.xpath("//div[@class='breadcrumbs breadcrumbs_article js-ago-wrapper']")
        for item1 in items1:
            date_news = item1.xpath(".//span[@class='note__text breadcrumbs__text js-ago']/text()")
            source_news = item1.xpath(".//span[@class='link__text']/text()")
            new['date'] = date_news[0]
            new['source'] = source_news[0]

        news.append(new)
        with open('news_mailru.json', 'w') as f:
            json.dump(news, f)
    return news

In [ ]:
ef request_to_lentaru():
    main_link = 'https://lenta.ru'
    response = requests.get('https://lenta.ru/rubrics/economics/',
                            headers=header)
    dom = html.fromstring(response.text)
    news = []
    items = dom.xpath("//div[@class='row js-content']//div[@class='item news b-tabloid__topic_news']")
    for item in items:
        new = {}
        name = item.xpath(".//h3//span/text()")
        link = item.xpath(".//h3//a/@href")
        date_news = item.xpath(".//span[@class='g-date item__date']/text()")
        new['name'] = name[0].replace('\xa0', ' ')
        new['link'] = main_link + link[0]
        new['date'] = date_news[0]
        new['source'] = 'Lenta.ru'
        news.append(new)
        with open('news_lentaru.json', 'w') as f:
            json.dump(news, f)
    return news

In [ ]:
pprint(request_to_yandex())
pprint(request_to_mailru())
pprint(request_to_lentaru())

# Загрузка новостей в базу данных

In [ ]:
from pymongo import MongoClient
import pymongo
import pprint
import json
from pymongo.errors import DuplicateKeyError, BulkWriteError

In [ ]:
with open('news_yandex.json', encoding='utf-8') as f:
    news_yandex_list = json.load(f)
with open('news_mailru.json', encoding='utf-8') as f:
    news_mailru_list = json.load(f)
with open('news_lentaru.json', encoding='utf-8') as f:
    news_lentaru_list = json.load(f)

In [ ]:
client = MongoClient('localhost', 27017)
db = client['news_db']
news_yandex = db.news_yandex
news_mailru = db.news_mailru
news_lentaru = db.news_lentaru

In [ ]:
def write_to_base(news_list, collections):
    j = 0
    k = 0
    for i in range(0, len(news_list)):
        try:
            collections.insert_one(news_list[i])
            j += 1
        except (DuplicateKeyError):
            k += 1
    print(f'В базу записано {j} уникальных документов')
    print(f'Выявлено {k} дубликтов документов')

In [ ]:
write_to_base(news_yandex_list, news_yandex)
write_to_base(news_mailru_list, news_mailru)
write_to_base(news_lentaru_list, news_lentaru)